In [1]:
# Donchian Weekly Trend Following System
import gta_indicator
import gta_prices
import gta_signals
import numpy as np
import pandas as pd

In [2]:
# Trade parameters.
exchange = 'LSE'
tidm = 'IBPO'
timeframe = 'Weekly'
filename = f'{exchange}_{tidm}_prices.csv'
p1 = 48 # System 1 look back period.
p2 = 24 # System 2 look back period.
p3 = 12 # System 3 look back period.
p4 = 6  # System 4 look back period.
position_size = 7500  # Position size in major currency unit.
risk_pct = 0.2 # Percentage risk per trade.
charges = 11.95  # Commission per trade.
sduty = 0.0  # Stamp Duty percentage.

In [3]:
# Import weekly closing prices.
prices = gta_prices.weekly(exchange, tidm)

In [4]:
# Donchian channels.
dc1 = gta_indicator.donchian(prices, p1)
dc2 = gta_indicator.donchian(prices, p2)
dc3 = gta_indicator.donchian(prices, p3)
dc4 = gta_indicator.donchian(prices, p4)

In [5]:
# System 1 entries & exits.
s1 = pd.concat([prices, dc1], axis=1)
s1['sys'] = 1
s1['buy'] = np.where(s1.close > s1.upr, 1, 0)
s1['sell'] = np.where(s1.close < s1.mid, 1, 0)
s1['state'] = gta_signals.state_signal(s1.buy, s1.sell, p1)
s1['entry'] = np.where(np.logical_and(s1.state == 1, s1.state.shift(periods=1) == 0), 1, 0)
s1['exit'] = np.where(np.logical_and(s1.state == 0, s1.state.shift(periods=1) == 1), 1, 0)

In [6]:
# System 2 entries & exits.
s2 = pd.concat([prices, dc2], axis=1)
s2['sys'] = 2
s2['buy'] = s1.entry
s2['sell'] = np.where(s2.close < s2.mid, 1, 0)
s2['state'] = gta_signals.state_signal(s2.buy, s2.sell, p2)
s2['entry'] = np.where(np.logical_and(s2.state == 1, s2.state.shift(periods=1) == 0), 1, 0)
s2['exit'] = np.where(np.logical_and(s2.state == 0, s2.state.shift(periods=1) == 1), 1, 0)

In [7]:
# System 3 entries & exits.
s3 = pd.concat([prices, dc3], axis=1)
s3['sys'] = 3
s3['buy'] = s1.entry
s3['sell'] = np.where(s3.close < s3.mid, 1, 0)
s3['state'] = gta_signals.state_signal(s3.buy, s3.sell, p3)
s3['entry'] = np.where(np.logical_and(s3.state == 1, s3.state.shift(periods=1) == 0), 1, 0)
s3['exit'] = np.where(np.logical_and(s3.state == 0, s3.state.shift(periods=1) == 1), 1, 0)

In [8]:
# System 4 entries & exits.
s4 = pd.concat([prices, dc4], axis=1)
s4['sys'] = 4
s4['buy'] = s1.entry
s4['sell'] = np.where(s4.close < s4.mid, 1, 0)
s4['state'] = gta_signals.state_signal(s4.buy, s4.sell, p4)
s4['entry'] = np.where(np.logical_and(s4.state == 1, s4.state.shift(periods=1) == 0), 1, 0)
s4['exit'] = np.where(np.logical_and(s4.state == 0, s4.state.shift(periods=1) == 1), 1, 0)

In [9]:
# All entries & exits.
td = pd.concat([s1[s1.entry == 1] , s1[s1.exit == 1], s2[s2.exit == 1], s3[s3.exit == 1], s4[s4.exit == 1]], axis=0)
td = td.sort_index()

In [10]:
# Reindex by trade.
td['trade'] = td.state.cumsum()
td = td.reset_index()
td = td.set_index('trade')

In [11]:
# Position size (buy).
td['volatility'] = np.where(td.entry == 1, abs((td.mid - td.close) / td.close).round(3), 0)

# td['risk_amt'] = np.where(td.entry == 1, ((position_size * risk_pct) / td.volatility).round(2), 0)
# td['shares'] = np.where(td.entry == 1, (td.risk_amt / td.close).astype('int'), 0)
td

,date,open,high,low,close,upr,lwr,mid,sys,buy,sell,state,entry,exit,volatility
trade,,,,,,,,,,,,,,,
1,2011-08-26,2.2100,2.2960,2.210,2.260,2.246,1.3500,1.79800,1,1,0,1,1,0,0.204
1,2011-10-14,2.3250,2.3250,2.325,2.325,2.400,2.2600,2.33000,4,0,1,0,0,1,0.000
1,2012-06-01,3.5750,3.5750,3.350,3.450,3.700,3.4000,3.55000,3,0,1,0,0,1,0.000
1,2013-03-29,3.6500,3.6500,3.625,3.625,3.800,3.6000,3.70000,2,0,1,0,0,1,0.000
1,2013-08-02,3.6250,3.6250,3.510,3.510,3.800,3.4000,3.60000,1,0,1,0,0,1,0.000
2,2016-05-13,0.4000,0.4200,0.400,0.420,0.400,0.1075,0.25375,1,1,0,1,1,0,0.396
2,2016-06-24,0.4650,0.4650,0.425,0.440,0.490,0.4000,0.44500,4,0,1,0,0,1,0.000
2,2017-02-17,0.8200,0.8200,0.820,0.820,0.835,0.8100,0.82250,3,0,1,0,0,1,0.000
2,2017-03-10,0.8200,0.8200,0.775,0.775,0.835,0.7700,0.80250,2,0,1,0,0,1,0.000


In [12]:
for i in range(len(td.index)):
    if td.volatility.iloc[i] == 0:
        td.risk_amt.iloc[i] = 200
        
td

AttributeError: 'DataFrame' object has no attribute 'risk_amt'